<a href="https://colab.research.google.com/github/pchodon/Amegashita/blob/Pain-Inhibition/main_v5.6%20code%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')




Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import multiprocessing
import time
from csv import reader
from google.colab import files
import pandas as pd
import threading
import os
import tensorflow as tf
import numpy as np
import sys
import matplotlib.pyplot as plt


from tensorflow import feature_column
#from tensorflow.keras import layers
from sklearn.model_selection import train_test_split


No_of_records = 4
Parameters = 4
No_of_elements = 10

x1 = tf.zeros([No_of_records,Parameters,No_of_elements])
np.b =np.zeros((No_of_records,Parameters,No_of_elements))
ta = tf.TensorArray(tf.float32, size=sys.getsizeof(x1), dynamic_size=True, clear_after_read=False)
rank_3_tensor= tf.convert_to_tensor(np.b, dtype=tf.float32)




sensory_1_resp = 0.5  # default 1 Sec
sensory_2_resp = 0.5  # default 1 Sec
temporal_resp =  1  # default 1 Sec
thalamus_resp =  1  # default 1 Sec
reset = 0
forever = 1
max_sensory_stream = 80
Max_lapse = 5
mTrue = 1
mFalse = 0
w, h = 10, 10;
hipp_first = False
stop_emulation = 0
Start_Sys_window = 12
window_start = 0

row,column = (4,10)
inhibit = []

# row 1 = time
# row 2 = sensor_1
# row_3 = sensor_1
# row_4 = learning

#101213141516171819
#20212223242526272829
#3031323334353637383930

TIME = 1
SENSOR_1 = 2
SENSOR_2 = 3
LEARNING = 4


# send_1_sensor,recv_1_sensor
# send_2_sensor,recv_2_senor

def sensory_1(send_1_sensor,recv_1_sensor,parent_conn,msg):
   
  data = []
  with open('/content/drive/MyDrive/sense_1.csv', 'r') as data:
    df_sensory_1 = pd.read_csv(data)

  i = 0
  while(1):
   msg = df_sensory_1.loc[i]
   send_1_sensor.send(msg[0])
   time.sleep(sensory_1_resp)
   i = i+1
   if(i==max_sensory_stream):
        break
  return  

def sensory_2(send_2_sensor,recv_2_sensor,parent_conn,msg):
  
  data = []
  with open('/content/drive/MyDrive/sense_2.csv', 'r') as data:
    df_sensory_2 = pd.read_csv(data)

  i = 0
  while(1):
   msg = df_sensory_2.loc[i]
   send_2_sensor.send(msg[0])
   time.sleep(sensory_2_resp)
   i = i+1
   if(i==max_sensory_stream):
        break
  return  

def hippocampus(xparent_conn,xchild_conn,msg):
    #print("Joke")
    while(1):
      msg = xchild_conn.recv()
      rank_3_tensor= tf.convert_to_tensor(msg, dtype=tf.float32)

      ##print("________Hippocampus reporting____________________",rank_3_tensor)

     
    return

def Start_Sys(Start_Sys_state_recv,msg):
   # Prearing the systemt to start functioning
   with open('/content/drive/MyDrive/sense_3.csv', 'r') as data:
    df_sensory_2 = pd.read_csv(data)

    i = 0
    time.sleep(2)
    msg = 9
    msg = df_sensory_2.loc[i]
    print("System getting ready...pls wait")
    Start_Sys_state_send.send(msg[0])
    


def thalamus(b,Start_Sys_state_send,recv_1_sensor,recv_2_sensor,parent_conn,child_conn,xparent_conn,xchild_conn,msg):
  maxim = 10
  lapse = 0
  window_start = 0
  window = False
  source = False
  sense = 0
  sensor_1,sensor_2 = [3,10]
  # row 1 time
  # row 2 sensor_1
  
  # row 3 sensor_2
  # Episode is 10 frames
  think_pause = 0
 
  episode_count = 0
  experience_count = 0

  if(window_start == 0):
        sense = Start_Sys_state_recv.recv()
        ##print(sense)
        if((sense == 9)):
         ##print("________Window starts here____________________")
         No_of_records = 4
         Parameters = 4
         No_of_elements = 10
         np.b =np.zeros((No_of_records,Parameters,No_of_elements))
         No_of_records = 4
         Parameters = 4
         No_of_elements = 10
         episode_count = 0
         window_start = 1
         experience_count = 0
         rank_3_tensor= tf.convert_to_tensor(np.b, dtype=tf.float32)
         inhibitor = 0
         
  while(1):
    #time.sleep(temporal_resp)
    sense_1 = recv_1_sensor.recv()
    ###print('sense 1 : ',sense_1)
    sense_2 = recv_2_sensor.recv()
    ###print('sense 2 : ',sense_2)
    mag = [0,1]
    if((sense_2 == 9)): 
      print("__________Start thinking")
      think_pause = 0
      xparent_conn.send(rank_3_tensor)
      episode_count = 0
      time.sleep(temporal_resp)
      print(".")
      continue

    #print("S1 S2 Exp, Epi\n",sense_1,sense_2,experience_count,episode_count)
      

    if((sense_2 == 'x') or (sense_2 =='x')):  
     ##print("__________No action__________________")
     continue                        
    

    if((sense_1 == '1') or (sense_1 =='0')):
      np.b[experience_count][0][episode_count] = sense_1

    if((sense_2 == '0') or (sense_2 =='1')):  
      np.b[experience_count][1][episode_count] = sense_2


    msg = [0,1]

    if((sense_1 == '1') and (sense_2 =='1')):  
      np.b[experience_count][2][episode_count] = 1
      np.b[experience_count][3][episode_count] = episode_count

      Start_Sys_state_send.send(msg[1])
      inhibitor = inhibitor + 1
   
    episode_count +=1
    if(episode_count >= 10):
      experience_count +=1
      rank_3_tensor= tf.convert_to_tensor(np.b, dtype=tf.float32)
      print("________Single episode____________________",experience_count)
      if(experience_count == 4):
        while(1):
          msg = rank_3_tensor#[0,5]
          start_timer.send(msg)
          time.sleep(5)
        return
      episode_count = 0
         
# Switching to autonomous simple fall of the edge

def Auto_swt(start_timer,stop_timer,msg):
  while(1):
    msg = stop_timer.recv() 
    print("Autonomous state process",msg)
    break
  while(1):  
    time.sleep(0.1)
    print("Sleeping....")  


if __name__ == "__main__":
  ##print("System Starting......")
  msg = 0

        # creating common pipe name
  parent_conn, child_conn = multiprocessing.Pipe()
  send_1_sensor,recv_1_sensor = multiprocessing.Pipe()
  send_2_sensor,recv_2_sensor = multiprocessing.Pipe()
  Start_Sys_state_send,Start_Sys_state_recv = multiprocessing.Pipe()
  xparent_conn, xchild_conn = multiprocessing.Pipe()
  timer = multiprocessing.Pipe()

  start_timer,stop_timer = multiprocessing.Pipe()

  p1 = multiprocessing.Process(target=sensory_1, args=(send_1_sensor,recv_1_sensor,parent_conn,msg))
  p2 = multiprocessing.Process(target=sensory_2, args=(send_2_sensor,recv_2_sensor,parent_conn,msg))
  p3 = multiprocessing.Process(target=thalamus, args=(np.b,Start_Sys_state_send,recv_1_sensor,recv_2_sensor,parent_conn,child_conn,xparent_conn, xchild_conn,msg))
  p4 = multiprocessing.Process(target=hippocampus, args=(xparent_conn,xchild_conn,msg))
  p5 = multiprocessing.Process(target=Start_Sys, args=(Start_Sys_state_recv,msg))
  p6 = multiprocessing.Process(target=Auto_swt, args=(start_timer,stop_timer,msg))

       # running processes

  p1.start();  p2.start();  p3.start();  p4.start();  p5.start();  p6.start()
  time.sleep(100)
  p6.join();  p5.join();  p4.join();  p3.join();  p2.join();  p1.join()
  time.sleep(2)
  print("Starting new episode")
#333444

Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....
Sleeping....

KeyboardInterrupt: ignored


Sleeping....
Sleeping....


Process Process-21:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-7-c40a3ca40ecb>", line 210, in thalamus
    time.sleep(5)
KeyboardInterrupt
Process Process-24:
Process Process-22:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-7-c40a3ca40ecb>", line 222, in Auto_swt
    time.sleep(0.1)
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._k